In [1]:
using DifferentialEquations
using ParameterizedFunctions
using DataFrames
using CSV
using Sundials

In [2]:
#####################
##Non-ODE Functions##
#####################

function temp_gen(hour)
    283 + hour / (20 * 24 * 365) + 10 * sin((2 * pi / 24) * hour) + 10 * sin((2 * pi / (24 * 365)) * hour)
end

function I_S(hour)
    0.001 + 0.0005 * sin((2 * pi / (24 * 365)) * hour) #Exogenous SOC input
end

function I_D(hour)
    0.0001 + 0.00005 * sin((2 * pi / (24 * 365)) * hour) #Exogenous DOC input
end

function arrhenius_temp_dep(parameter, temp, Ea, temp_ref)
    decayed_parameter = parameter * exp(-Ea / 0.008314 * (1 / temp - 1 / temp_ref))
end

function linear_temp_dep(parameter, temp, Q, temp_ref)
    modified_parameter = parameter - Q * (temp - temp_ref)
end

function analytical_steady_state(SOC_input, DOC_input, u_M, a_SD, a_DS, a_M, a_MSC, k_S_ref, k_D_ref, k_M_ref)
    D₀ = (DOC_input + SOC_input * a_SD) / (u_M + k_D_ref + u_M * a_M * (a_MSC - 1 - a_MSC * a_SD) - a_DS * k_D_ref * a_SD)
    S₀ = (SOC_input + D₀ * (a_DS * k_D_ref + u_M * a_M * a_MSC)) / k_S_ref
    M₀ = u_M * D₀ / k_M_ref
    return [S₀, D₀, M₀]
end

analytical_steady_state (generic function with 1 method)

In [3]:
########################
###Non-param Constants##
########################

temp_ref = 283

#########################
##ODE System Parameters##
#########################

u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

50

In [4]:
###############
##ODE Solving##
###############

tspan = (0., 100000.) #in hours
C₀ = analytical_steady_state(I_S(0), I_D(0), u_M, a_SD, a_DS, a_M, a_MSC, k_S_ref, k_D_ref, k_M_ref)
print(C₀)
p = [u_M, a_SD, a_DS, a_M, a_MSC, k_S_ref, k_D_ref, k_M_ref, Ea_S, Ea_D, Ea_M]

function CON!(du, u, p, t)
    S, D, M = u
    u_M, a_SD, a_DS, a_M, a_MSC, k_S_ref, k_D_ref, k_M_ref, Ea_S, Ea_D, Ea_M = p
    du[1] = dS = I_S(t) + a_DS * arrhenius_temp_dep(k_D_ref, temp_gen(t), Ea_D, temp_ref) * D + a_M * a_MSC * arrhenius_temp_dep(k_M_ref, temp_gen(t), Ea_M, temp_ref) * M - arrhenius_temp_dep(k_S_ref, temp_gen(t), Ea_S, temp_ref) * S
    du[2] = dD = I_D(t) + a_SD * arrhenius_temp_dep(k_S_ref, temp_gen(t), Ea_S, temp_ref) * S + a_M * (1 - a_MSC) * arrhenius_temp_dep(k_M_ref, temp_gen(t), Ea_M, temp_ref) * M - u_M * D - arrhenius_temp_dep(k_D_ref, temp_gen(t), Ea_D, temp_ref) * D
    du[3] = dM = u_M * D - arrhenius_temp_dep(k_M_ref, temp_gen(t), Ea_M, temp_ref) * M  
end

CON_prob = ODEProblem(CON!, C₀, tspan, p) #save result every 2 hour(s)
CON_sol = solve(CON_prob, CVODE_BDF(), dt = 0.05, saveat = 0:2:100000)

[45.66033972675598, 0.07146893594388856, 0.7146893594388857]

retcode: Success
Interpolation: 1st order linear
t: 50001-element Vector{Float64}:
      0.0
      2.0
      4.0
      6.0
      8.0
     10.0
     12.0
     14.0
     16.0
     18.0
     20.0
     22.0
     24.0
      ⋮
  99978.0
  99980.0
  99982.0
  99984.0
  99986.0
  99988.0
  99990.0
  99992.0
  99994.0
  99996.0
  99998.0
 100000.0
u: 50001-element Vector{Vector{Float64}}:
 [45.66033972675598, 0.07146893594388856, 0.7146893594388857]
 [45.65921023359436, 0.071654180033982, 0.714597470908103]
 [45.656643573614126, 0.0720979219281527, 0.7144028302543975]
 [45.65300286202577, 0.07273843083577423, 0.7141381096596483]
 [45.649494385293316, 0.0733392327742783, 0.7138835249001273]
 [45.64722023385764, 0.07369987030681226, 0.713715073675263]
 [45.6464472562167, 0.07380118192426902, 0.7136626104170748]
 [45.64680826966161, 0.07373810825560598, 0.7137110935862316]
 [45.647818684849085, 0.07360099980489838, 0.7138260691141065]
 [45.649112508577396, 0.07343867380586513, 0.7139724598373467]


In [5]:
tspan_steps = collect(0:1:100000)

100001-element Vector{Int64}:
      0
      1
      2
      3
      4
      5
      6
      7
      8
      9
     10
     11
     12
      ⋮
  99989
  99990
  99991
  99992
  99993
  99994
  99995
  99996
  99997
  99998
  99999
 100000

In [6]:
temp_array = zeros(length(tspan_steps))
for i = 1:length(tspan_steps)
    temp_array[i] = temp_gen(tspan_steps[i])
end

In [7]:
df_CON_soln = DataFrame(CON_sol.u, :auto)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,45.6603,45.6592,45.6566,45.653,45.6495,45.6472,45.6464,45.6468
2,0.0714689,0.0716542,0.0720979,0.0727384,0.0733392,0.0736999,0.0738012,0.0737381
3,0.714689,0.714597,0.714403,0.714138,0.713884,0.713715,0.713663,0.713711


In [8]:
CSV.write("CON_synthetic_soln.csv", df_CON_soln)

"CON_synthetic_soln.csv"

In [9]:
df_temp = DataFrame(hour = tspan_steps[1:2:end], temp = temp_array[1:2:end]) #records temps every 2 hour(s)
CSV.write("synthetic_temp.csv", df_temp)

"synthetic_temp.csv"

In [10]:
function CON_ss!(du, u, p, t)
    S, D, M = u
    u_M, a_SD, a_DS, a_M, a_MSC, k_S_ref, k_D_ref, k_M_ref, Ea_S, Ea_D, Ea_M = p
    du[1] = dS = I_S(t) + a_DS * arrhenius_temp_dep(k_D_ref, temp_ref, Ea_D, temp_ref) * D + a_M * a_MSC * arrhenius_temp_dep(k_M_ref, temp_ref, Ea_M, temp_ref) * M - arrhenius_temp_dep(k_S_ref, temp_ref, Ea_S, temp_ref) * S
    du[2] = dD = I_D(t) + a_SD * arrhenius_temp_dep(k_S_ref, temp_ref, Ea_S, temp_ref) * S + a_M * (1 - a_MSC) * arrhenius_temp_dep(k_M_ref, temp_ref, Ea_M, temp_ref) * M - u_M * D - arrhenius_temp_dep(k_D_ref, temp_ref, Ea_D, temp_ref) * D
    du[3] = dM = u_M * D - arrhenius_temp_dep(k_M_ref, temp_ref, Ea_M, temp_ref) * M  
end

CON_ss_prob = ODEProblem(CON_ss!, C₀, tspan, p)
CON_ss_sol = solve(CON_ss_prob)

retcode: Success
Interpolation: automatic order switching interpolation
t: 231-element Vector{Float64}:
      0.0
      9.999999999999999e-5
      0.0010999999999999998
      0.011099999999999997
      0.11109999999999996
      1.1110999999999995
     11.111099999999993
     75.20515757091302
    186.10787014516467
    323.22932192984376
    508.43799348843754
    745.6116721367242
   1101.976978805225
      ⋮
  95248.90384934294
  95669.62809191132
  96097.55479171284
  96538.5402198793
  97003.03479480553
  97522.23135813742
  98069.54556829006
  98511.33358081257
  98943.67233873373
  99375.56069923908
  99806.53500399168
 100000.0
u: 231-element Vector{Vector{Float64}}:
 [45.66033972675598, 0.07146893594388856, 0.7146893594388857]
 [45.66033972675598, 0.07146893594388874, 0.7146893594388857]
 [45.66033972675619, 0.07146893594391027, 0.7146893594388857]
 [45.66033972677807, 0.07146893594609785, 0.7146893594388857]
 [45.660339728969305, 0.07146893616516371, 0.714689359438902]
 [45.66